In [2]:
import geopandas as gpd
import os
import zipfile as zf
import geopandas
from shapely.geometry import shape
import gzip 
import urllib.request
import xarray as xr
import io
import shapefile

In [3]:
zp = zf.ZipFile('TM_WORLD_BORDERS-0.3.zip')
files_to_read = [y for y in zp.namelist() for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
dummy = zp.read(files_to_read[0])
dbf_file, prj_file, shp_file, shx_file = [zp.open(filename) for filename in files_to_read]
r = shapefile.Reader(shp = shp_file, shx = shx_file, dbf = dbf_file, encoding='windows-1252')
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]
for row in r.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names, row.record)))

In [4]:
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)

In [7]:
gdf

,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry
0,AC,AG,ATG,28,Antigua and Barbuda,44,83039,19,29,-61.783,17.078,"MULTIPOLYGON (((-61.68667 17.02444, -61.73806 ..."
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ..."
2,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."
3,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.43621 41.02107, 19.45055 41.06000..."
4,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"MULTIPOLYGON (((45.57305 40.63249, 45.52888 40..."
...,...,...,...,...,...,...,...,...,...,...,...,...
241,TB,BL,BLM,652,Saint Barthelemy,0,0,19,29,-63.043,18.040,"POLYGON ((-63.02834 18.01555, -63.03334 18.015..."
242,GK,GG,GGY,831,Guernsey,0,0,150,154,-2.576,49.459,"POLYGON ((-2.59083 49.42249, -2.59722 49.42249..."
243,JE,JE,JEY,832,Jersey,0,0,150,154,-2.129,49.219,"POLYGON ((-2.01500 49.21416, -2.02111 49.17722..."
244,SX,GS,SGS,239,South Georgia South Sandwich Islands,0,0,0,0,-36.891,-54.209,"MULTIPOLYGON (((-27.32584 -59.42722, -27.29806..."


Scraping does not work on my laptop now :

In [ ]:
 url = "https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz"
req = urllib.request.Request(url)
with gzip.open(urllib.request.urlopen(req)) as resp:
    xr_df = xr.open_dataset(io.BytesIO(resp.read()))
df = xr_df.to_dataframe()
dfnasa = df.reset_index()

In [ ]:
gdf2 = df2[df2['time'].dt.year > 2000]
del df2
del df
gdf2 = gpd.GeoDataFrame(
    gdf2, geometry=gpd.points_from_xy(gdf2.lat, gdf2.lon))

In [7]:
nasa50 = dfnasa[dfnasa['time'].dt.year > 1950]
nasa50 = gpd.GeoDataFrame(
    nasa50, geometry=gpd.points_from_xy(nasa50.lat, nasa50.lon))

NameError: name 'dfnasa' is not defined

In [7]:
gdf2

,lat,lon,nv,time,time_bnds,tempanomaly,geometry
1632,-89.0,-179.0,0,2016-01-15,2016-01-01,-1.23,POINT (-89.000 -179.000)
1633,-89.0,-179.0,0,2016-02-15,2016-02-01,-1.38,POINT (-89.000 -179.000)
1634,-89.0,-179.0,0,2016-03-15,2016-03-01,-0.01,POINT (-89.000 -179.000)
1635,-89.0,-179.0,0,2016-04-15,2016-04-01,0.39,POINT (-89.000 -179.000)
1636,-89.0,-179.0,0,2016-05-15,2016-05-01,0.51,POINT (-89.000 -179.000)
...,...,...,...,...,...,...,...
54950395,89.0,179.0,1,2020-12-15,2021-01-01,4.75,POINT (89.000 179.000)
54950396,89.0,179.0,1,2021-01-15,2021-02-01,8.34,POINT (89.000 179.000)
54950397,89.0,179.0,1,2021-02-15,2021-03-01,6.73,POINT (89.000 179.000)
54950398,89.0,179.0,1,2021-03-15,2021-04-01,3.72,POINT (89.000 179.000)


In [10]:
final_gdf = gpd.sjoin(gdf, gdf2, how="inner")

In [11]:
final_gdf

,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry,index_right,lat,lon,nv,time,time_bnds,tempanomaly
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",27841492,1.0,35.0,1,2017-09-15,2017-10-01,2.20
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",27841493,1.0,35.0,1,2017-10-15,2017-11-01,1.08
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",27841494,1.0,35.0,1,2017-11-15,2017-12-01,0.62
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",27841495,1.0,35.0,1,2017-12-15,2018-01-01,0.82
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ...",27841496,1.0,35.0,1,2018-01-15,2018-02-01,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,SV,SJ,SJM,744,Svalbard,0,0,150,154,18.374,78.830,"MULTIPOLYGON (((-8.29089 70.97415, -8.39447 70...",32795529,17.0,77.0,0,2019-06-15,2019-06-01,1.68
239,SV,SJ,SJM,744,Svalbard,0,0,150,154,18.374,78.830,"MULTIPOLYGON (((-8.29089 70.97415, -8.39447 70...",32795530,17.0,77.0,0,2019-07-15,2019-07-01,0.98
239,SV,SJ,SJM,744,Svalbard,0,0,150,154,18.374,78.830,"MULTIPOLYGON (((-8.29089 70.97415, -8.39447 70...",32795531,17.0,77.0,0,2019-08-15,2019-08-01,0.53
239,SV,SJ,SJM,744,Svalbard,0,0,150,154,18.374,78.830,"MULTIPOLYGON (((-8.29089 70.97415, -8.39447 70...",32795532,17.0,77.0,0,2019-09-15,2019-09-01,0.24
